<a href="https://colab.research.google.com/github/ajuhz/Artificial-Intelligence/blob/master/Variational_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image # to save the reconstrcted image in tensor

In [52]:
#define hyperparameter
image_size=784 #MNIST dataset image size 28x28 
hidden_dim = 400
latent_dim = 20
batch_size = 128
epochs = 10


In [53]:
#setting device GPU/CPU 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [54]:
#loading dataset and creating dataloader
train_dataset=torchvision.datasets.MNIST('../../Data',
                                         train=True,
                                         transform=transforms.ToTensor(),
                                         download=True)
test_dataset=torchvision.datasets.MNIST('../../Data',
                                         train=False,
                                         transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_dataset,
                                                 batch_size=batch_size,
                                                 shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                                 batch_size=batch_size,
                                                 shuffle=True)


In [62]:
#create a directory for the result image
result_dir='results'
if not os.path.exists(result_dir):
  os.makedirs(result_dir)


**Variational Autoencoder**
![vae](https://user-images.githubusercontent.com/30661597/78418103-a2047200-766b-11ea-8205-c7e5712715f4.png)



In [56]:
# Defining th model
class VAE(nn.Module):
  def __init__(self):
    super(VAE,self).__init__()
    self.fc1=nn.Linear(image_size,hidden_dim)
    self.fc2_mean=nn.Linear(hidden_dim,latent_dim)
    self.fc2_std=nn.Linear(hidden_dim,latent_dim)
    self.fc3=nn.Linear(latent_dim,hidden_dim)
    self.fc4=nn.Linear(hidden_dim,image_size)
  
  def encode(self,x):
    h=F.relu(self.fc1(x))
    mu = self.fc2_mean(h)
    logvar = self.fc2_std(h)
    return mu,logvar
  
  def reparameterize(self, mu, logvar):
    std = torch.exp(logvar/2)
    eps = torch.randn_like(std)
    return (mu + eps * std)
  
  def decode(self,z):
    h= F.relu(self.fc3(z))
    out = torch.sigmoid(self.fc4(h))
    return out

  def forward(self,x):
    mu,logvar=self.encode(x.view(-1, image_size))
    z=self.reparameterize(mu,logvar)
    reconstructed=self.decode(z)
    return reconstructed, mu, logvar


In [57]:
# create model and optimizer obejct
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

$Loss = -E[\log P(X | z)]+D_{K L}[N(\mu(X), \Sigma(X)) \| N(0,1)]$
#### $D_{K L}[N(\mu(X), \Sigma(X)) \| N(0,1)]=\frac{1}{2} \sum_{k}\left(\exp (\Sigma(X))+\mu^{2}(X)-1-\Sigma(X)\right)$

In [58]:
#define loss function
def loss_fn(reconstructed,original,mu,logvar):
  rc_loss = F.binary_cross_entropy(reconstructed,original.view(-1,image_size),reduction='sum')
  kld= 0.5*(torch.sum(logvar.exp()+mu.pow(2)-1-logvar))
  return rc_loss + kld

In [59]:
# Function to Train the model 
def train(epochs):
  model.train()
  train_loss=0
  for i,(images,_) in enumerate(train_loader):
    images=images.to(device)
    reconstructed, mu, logvar = model(images)
    loss=loss_fn(reconstructed,images,mu,logvar)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%100==0:
      print("Train Epoch {} [Batch {}/{}]\tLoss: {:.3f}".format(epoch, i, len(train_loader), loss.item()/len(images)))
  print('=====> Epoch {}, Average Loss: {:.3f}'.format(epoch, train_loss/len(train_loader.dataset)))

In [60]:
#Test the model
# Test function
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (images, _) in enumerate(test_loader):
            images = images.to(device)
            reconstructed, mu, logvar = model(images)
            test_loss += loss_fn(reconstructed, images, mu, logvar).item()
            if batch_idx == 0:
                comparison = torch.cat([images[:5], reconstructed.view(batch_size, 1, 28, 28)[:5]])
                save_image(comparison.cpu(), 'results/reconstruction_' + str(epoch) + '.png', nrow = 5)

    print('=====> Average Test Loss: {:.3f}'.format(test_loss/len(test_loader.dataset)))

In [63]:
# Main function
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        # Get rid of the encoder and sample z from the gaussian ditribution and feed it to the decoder to generate samples
        sample = torch.randn(64,20).to(device)
        generated = model.decode(sample).cpu()
        save_image(generated.view(64,1,28,28), 'results/sample_' + str(epoch) + '.png')

Train Epoch 1 [Batch 0/469]	Loss: 125.698
Train Epoch 1 [Batch 100/469]	Loss: 124.848
Train Epoch 1 [Batch 200/469]	Loss: 123.126
Train Epoch 1 [Batch 300/469]	Loss: 120.954
Train Epoch 1 [Batch 400/469]	Loss: 115.409
=====> Epoch 1, Average Loss: 121.633
=====> Average Test Loss: 115.901
Train Epoch 2 [Batch 0/469]	Loss: 114.493
Train Epoch 2 [Batch 100/469]	Loss: 115.269
Train Epoch 2 [Batch 200/469]	Loss: 113.379
Train Epoch 2 [Batch 300/469]	Loss: 117.440
Train Epoch 2 [Batch 400/469]	Loss: 113.625
=====> Epoch 2, Average Loss: 114.721
=====> Average Test Loss: 112.308
Train Epoch 3 [Batch 0/469]	Loss: 110.809
Train Epoch 3 [Batch 100/469]	Loss: 112.703
Train Epoch 3 [Batch 200/469]	Loss: 112.672
Train Epoch 3 [Batch 300/469]	Loss: 112.762
Train Epoch 3 [Batch 400/469]	Loss: 111.055
=====> Epoch 3, Average Loss: 111.736
=====> Average Test Loss: 109.838
Train Epoch 4 [Batch 0/469]	Loss: 112.761
Train Epoch 4 [Batch 100/469]	Loss: 111.337
Train Epoch 4 [Batch 200/469]	Loss: 114.400
